<a href="https://colab.research.google.com/github/reejungkim/Pipeline/blob/main/prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import libraries

In [10]:
import warnings
warnings.filterwarnings("ignore")

# Load up packages
import pandas as pd
import numpy as np

#visualization lisb
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_palette('Set2')


from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split,  cross_val_score
#preprocessing libs
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
#label encoders
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

#Models
#classifiers
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
#regressors
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor , GradientBoostingRegressor


#model evaluation libs
#classifier evaluation
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
#regressor evaluation
from sklearn.metrics import r2_score, mean_squared_error

#Hyper-parameter tuning
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


# Load data set

In [23]:
# Load data
df = pd.read_csv('bankruptcy_Train.csv')

# Checking for nulls and data types

In [18]:
# checking for sparse columns 
#(df.isnull().sum()/ len(df).sort_values(ascending=False)[:30]
df_null = pd.DataFrame((df.isnull().sum()/ len(df)), columns={"%missing"})
df_null.loc[df_null['%missing']>=.8]

,%missing


should check if the sparse columns refers to zero or n/a before dropping the columns
 

# Checking columns with small variations

In [25]:
#df.drop(['Attr1', 'Attr2'], axis=1)

In [50]:
df_no_var = df.loc[:, df.columns!='class'].std().sort_values(ascending=True)
df_no_var.loc[df_no_var<=0.01]

Attr41    0.004954
dtype: float64

# Imputer (replacing null values)

In [ ]:
# Impute null objects with mode
for col in ['Electrical', 'RoofStyle', 'Neighborhood']:
    df[col] = df[col].fillna(df[col].mode()[0])

In [28]:
cols_obj = df.select_dtypes(include=['object']).columns
#replace null objects with 'None'
for col in cols_obj:
    df[col] = df[col].fillna('None')

In [22]:
cols_number = df.select_dtypes(include=['int64', 'float64']).columns
# Null values of rest of numerical types variables would better be replaced by median of neighborhood group
for col in cols_number:
    all_data[col] = all_data.groupby('Neighborhood')[col].transform( lambda x: x.fillna(x.median()))

Index(['Attr1', 'Attr2', 'Attr3', 'Attr4', 'Attr5', 'Attr6', 'Attr7', 'Attr8',
       'Attr9', 'Attr10', 'Attr11', 'Attr12', 'Attr13', 'Attr14', 'Attr15',
       'Attr16', 'Attr17', 'Attr18', 'Attr19', 'Attr20', 'Attr21', 'Attr22',
       'Attr23', 'Attr24', 'Attr25', 'Attr26', 'Attr27', 'Attr28', 'Attr29',
       'Attr30', 'Attr31', 'Attr32', 'Attr33', 'Attr34', 'Attr35', 'Attr36',
       'Attr37', 'Attr38', 'Attr39', 'Attr40', 'Attr41', 'Attr42', 'Attr43',
       'Attr44', 'Attr45', 'Attr46', 'Attr47', 'Attr48', 'Attr49', 'Attr50',
       'Attr51', 'Attr52', 'Attr53', 'Attr54', 'Attr55', 'Attr56', 'Attr57',
       'Attr58', 'Attr59', 'Attr60', 'Attr61', 'Attr62', 'Attr63', 'Attr64',
       'class'],
      dtype='object')

# Vectorization

In [29]:
for col in cols_obj:
    print('{}: {}'.format(col, df[col].unique()  ))

In [ ]:
#Encoding object type variables using label encoder
for col in cols_obj:
    df[col] = le.fit_transform(df[col])

# Applying Scaler 

In [ ]:
#choose scaler
scaler = RobustScaler()

In [ ]:
#scaler application
scaled_X = scaler.fit_transform(all_data.loc[:, all_data.columns!='SalePrice'].reset_index(drop=True))
scaled_Y = scaler.fit_transform( all_data[0:1460][['SalePrice']] ) 

In [19]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored